In [ ]:
import sys
sys.path.append("/Users/chase/Desktop/Comp_Sci/Capstone/Dynasty")

from src.process_college import build_player_dict

player_dict = build_player_dict(verbose=False)
print(len(player_dict))
print(player_dict.get("Cameron Ward"))


In [ ]:
import sys
sys.path.append("/Users/chase/Desktop/Comp_Sci/Capstone/Dynasty")

from src.process_combine import build_combine_dict

player_combine = build_combine_dict(verbose=False)
print(len(player_combine))
print(player_combine.get("Amon-Ra St Brown"))


In [ ]:
from src.process_pro_qb import run_pro_qb_player

qb_dict = run_pro_qb_player(years=range(2016, 2025), s_type="REG", verbose=False)
print(len(qb_dict))
print(qb_dict.get("Patrick Mahomes"))

In [ ]:
from src.process_pro_wr import run_pro_wr_player

wr_dict = run_pro_wr_player(years=range(2016, 2025), s_type="REG", verbose=False)
print(len(wr_dict))
print(wr_dict.get("Rashee Rice"))

In [ ]:
from src.process_pro_rb import run_pro_rb_player

rb_dict = run_pro_rb_player(years=range(2016, 2025), s_type="REG", verbose=False)
print(len(rb_dict))
print(rb_dict.get("Christian McCaffrey"))

In [ ]:
from src.process_pro_te import run_pro_te_player

te_dict = run_pro_te_player(years=range(2016, 2025), s_type="REG", verbose=False)
print(len(te_dict))
print(te_dict.get("Travis Kelce"))

In [ ]:
from src.fantasycalc_client import (
        get_player_value, search_players,
        get_rankings_df, save_current_rankings
    )

# Look up one player
row = get_player_value("Breece Hall")
print(row)

# Search for possible name matches
print(search_players("Harrison"))

# Get full rankings as a DataFrame
df = get_rankings_df(dynasty=True, num_qbs=2, teams=12, ppr=1.0)
print(df.head())

# Save CSV snapshot(s) to Market_Value/
path = save_current_rankings(dynasty=True, num_qbs=2, teams=12, ppr=1.0)
print("Saved:", path)

In [ ]:
from src.plot_scatter_wr_combine import main
main(years=range(2020, 2026), limit=50, pick_min=1, pick_max=32)


In [ ]:
print(clean_player_name("Amon-Ra St. Brown"))   # Amon-Ra StBrown
print(clean_player_name("amon-ra st brown"))    # Amon-Ra StBrown
print(clean_player_name(None))                  # ""


In [ ]:
import re

def clean_player_name(player_name):
    """Clean names while preserving apostrophes/hyphens, fusing 'St. X' -> 'StX',
    and normalizing casing to Title Case."""
    if not isinstance(player_name, str):
        return player_name

    s = player_name.strip()

    # 1) Remove common suffixes (Jr, Sr, II, III, IV, V), case-insensitive
    suffixes = ['Jr', 'Sr', 'II', 'III', 'IV', 'V']
    s = re.sub(r'\b(?:' + '|'.join(suffixes) + r')\b\.?', '', s, flags=re.IGNORECASE)

    # 2) Keep only word chars, whitespace, apostrophes, and hyphens
    s = re.sub(r"[^\w\s'-]", '', s)

    # 3) Fuse 'St. ' or 'St ' (any case) before a capitalized surname -> 'StSurname'
    s = re.sub(r"\bSt[.\s]+(?=[A-Z])", "St", s, flags=re.IGNORECASE)

    # 4) Collapse extra spaces
    s = ' '.join(s.split())

    # 5) Normalize to Title Case (preserves apostrophes/hyphens properly)
    s = s.title()

    # Fix common cases where title-casing breaks (e.g., "O'Neal" -> "O'Neal", not "O'Neal")
    # The default .title() already does this okay, but just in case:
    s = re.sub(r"\bO'([A-Z])", lambda m: "O'" + m.group(1).upper(), s)

    return s


def strip_name_marks(s: object) -> object:
    """Strip common extraneous marks like '*' without touching apostrophes or hyphens."""
    if not isinstance(s, str):
        return s
    return s.replace("*", "")


# ---- quick checks ----
tests = [
    "Amon-Ra St. Brown",
    "amon-ra st. brown",
    "O'Neal Jr.",
    "jean-baptiste iii",
    "ST. JOHN",
]
for t in tests:
    print(t, "->", clean_player_name(t))


In [ ]:
import sys
sys.path.append("../src")   # path to src relative to your notebook

from scrape_nfldraftbuzz import parse_profile

url = "https://www.nfldraftbuzz.com/Player/Jaxson-Dart-QB-USC"
row = parse_profile(url)
row


In [1]:
import sys
import pandas as pd

# make sure Python can see your src/ folder
sys.path.append("../src")

from src.scrape_nfldraftbuzz import parse_profile

# Jaxson Dart’s DraftBuzz profile
url = "https://www.nfldraftbuzz.com/Player/Jaxson-Dart-QB-USC"

# scrape into a dict
row = parse_profile(url)
print(row)

# optional: convert to DataFrame for easier viewing
df = pd.DataFrame([row])
df.T   # show transposed for readability


[fetch] 403 Forbidden for https://www.nfldraftbuzz.com/Player/Jaxson-Dart-QB-USC
{'source_url': 'https://www.nfldraftbuzz.com/Player/Jaxson-Dart-QB-USC', 'player': None, 'position': None, 'college': None, 'class': None, 'height_raw': None, 'height_in': None, 'height_m': None, 'weight_raw': None, 'weight_lb': None, 'weight_kg': None, 'forty': None, 'ten_split': None, 'shuttle': None, 'three_cone': None, 'vertical': None, 'broad': None, 'bench': None, 'hand_size': None, 'arm_length': None, 'wingspan': None, 'age': None, 'dob': None, 'class_year': None, 'draft_projection': None, 'ras': None}


,0
source_url,https://www.nfldraftbuzz.com/Player/Jaxson-Dar...
player,None
position,None
college,None
class,None
height_raw,None
height_in,None
height_m,None
weight_raw,None
weight_lb,None


In [ ]:
from src.mockdraftable_scraper import build_mockdraftable_dataframe

player_names = [
    "Jaxson Dart",
    "Ashton Jeanty",
    "Marvin Harrison Jr",
]

search_pages = [
    # you can include multiple filters/positions/years and the code will merge results
    "https://www.mockdraftable.com/search?position=QB&beginYear=1999&endYear=2025&sort=DESC&page=1",
    "https://www.mockdraftable.com/search?position=RB&beginYear=1999&endYear=2025&sort=DESC&page=1",
    "https://www.mockdraftable.com/search?position=WR&beginYear=1999&endYear=2025&sort=DESC&page=1",
]

df = build_mockdraftable_dataframe(
    player_names=player_names,
    search_base_urls=search_pages,
    max_pages_per_base=10,     # crawl up to 10 pages per search url
)

df  # tidy, wide DataFrame of measurables + meta
